In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip'

In [4]:
zip_dir = tf.keras.utils.get_file('pizza_and_streak.zip', origin=file_url, extract=True)

109551616/109540975 [==============================] - 6s 0us/step


In [5]:
import pathlib

In [6]:
path = pathlib.Path(zip_dir).parent / 'pizza_steak'

In [8]:
train_dir = path / 'train'
validation_dir = path / 'test'

In [9]:
train_dir

WindowsPath('C:/Users/Bhara/.keras/datasets/pizza_steak/train')

In [10]:
train_pizza_dir = train_dir / 'pizza'
train_steak_dir = train_dir /'steak'
validation_pizza_dir = validation_dir / 'pizza'
validation_steak_dir = validation_dir / 'steak'

In [11]:
import os

In [14]:
total_train = len(os.listdir(train_pizza_dir)) + len(os.listdir(train_steak_dir))
total_val = len(os.listdir(validation_pizza_dir)) + len(os.listdir(validation_steak_dir))

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [18]:
batch_size = 16
img_height = 224
img_width = 224

In [21]:
train_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size,
                                                           directory = train_dir,
                                                           shuffle=True, 
                                                           target_size = (img_height, img_width),
                                                           class_mode='binary')

Found 1500 images belonging to 2 classes.


In [22]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size = batch_size,
                                                              directory = validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='binary')

Found 500 images belonging to 2 classes.


In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [24]:
np.random.seed(8)
tf.random.set_seed(8)

In [25]:
from tensorflow.keras.applications import NASNetMobile

In [26]:
base_model = NASNetMobile(include_top=False,
                          input_shape=(img_height, img_width, 3),
                          weights='imagenet')

20004864/19993432 [==============================] - 2s 0us/step


In [27]:
base_model.trainable = False

In [28]:
base_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                            

In [30]:
model = tf.keras.Sequential([base_model,layers.Flatten(),
                             layers.Dense(500, activation='relu'),
                             layers.Dense(1, activation='sigmoid')])

In [31]:
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(0.001), 
              metrics=['accuracy'])

In [32]:
model.fit(train_data_gen, 
          steps_per_epoch = total_train // batch_size, 
          epochs=5, 
          validation_data = val_data_gen, 
          validation_steps = total_val // batch_size)

Epoch 1/5
93/93 [==============================] - 33s 127ms/step - loss: 2.1069 - accuracy: 0.9292 - val_loss: 0.3294 - val_accuracy: 0.9839
Epoch 2/5
93/93 [==============================] - 8s 85ms/step - loss: 0.2617 - accuracy: 0.9825 - val_loss: 0.2514 - val_accuracy: 0.9819
Epoch 3/5
93/93 [==============================] - 8s 88ms/step - loss: 0.1199 - accuracy: 0.9885 - val_loss: 0.2131 - val_accuracy: 0.9859
Epoch 4/5
93/93 [==============================] - 8s 88ms/step - loss: 0.0665 - accuracy: 0.9926 - val_loss: 0.3573 - val_accuracy: 0.9879
Epoch 5/5
93/93 [==============================] - 8s 87ms/step - loss: 0.0997 - accuracy: 0.9912 - val_loss: 0.5353 - val_accuracy: 0.9819
